In [3]:
import importlib
import data_preprocessing
import data_set_and_loader
import models.multihead_pitch_prediction
from layers.lstm import LSTM
importlib.reload(data_preprocessing)
importlib.reload(data_set_and_loader)
importlib.reload(models.multihead_pitch_prediction)

<module 'models.multihead_pitch_prediction' from 'c:\\Users\\Richard\\Documents\\SEG4300\\Project\\SEG4300-Project\\Code\\models\\multihead_pitch_prediction.py'>

In [7]:
import torch
from torch import nn
import torch.optim as optim
from models.multihead_pitch_prediction import MultiHeadLSTM
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [ ]:
#NEED TO SHIFT CAT INDICES UP 1 AFTER ENCODING BECAUSE OF PADDING I THINK?

In [8]:
'''CATEGORY ENCODING INDEX
ADD 1 for PADDING
Pitch Type (y_type_encoder) - Total Unique Classes: 15
   AB -> 0 -> 1
   CH -> 1 -> 2
   CS -> 2 -> 3
   CU -> 3 -> 4
   EP -> 4 -> 5
   FA -> 5 -> 6
   FC -> 6 -> 7
   FF -> 7 -> 8
   FS -> 8 -> 9
   KC -> 9 -> 10
   KN -> 10 -> 11
   SI -> 11 -> 12
   SL -> 12 -> 13
   ST -> 13 -> 14
   SV -> 14 -> 15



   Result Description (y_desc_encoder) - Total Unique Classes: 13
   ball -> 0 -> 1
   blocked_ball -> 1 -> 2
   bunt_foul_tip -> 2 -> 3
   called_strike -> 3 -> 4
   foul -> 4 -> 5
   foul_bunt -> 5 -> 6
   foul_tip -> 6 -> 7
   hit_by_pitch -> 7 -> 8
   hit_into_play -> 8 -> 9
   intent_ball -> 9 -> 10
   missed_bunt -> 10 -> 11
   swinging_strike -> 11 -> 12
   swinging_strike_blocked -> 12 -> 13



   Result Event (y_event_encoder) - Total Unique Classes: 23
   IN_PROGRESS -> 0 -> 1
   catcher_interf -> 1 -> 2
   double -> 2 -> 2
   double_play -> 3 -> 4
   field_error -> 4 -> 5
   field_out -> 5 -> 6
   fielders_choice -> 6 -> 7
   fielders_choice_out -> 7 -> 8
   force_out -> 8 -> 9
   grounded_into_double_play -> 9 -> 10
   hit_by_pitch -> 10 -> 11
   home_run -> 11 -> 12
   sac_bunt -> 12 -> 13
   sac_bunt_double_play -> 13 -> 14
   sac_fly -> 14 -> 15
   sac_fly_double_play -> 15 -> 16
   single -> 16 -> 17
   strikeout -> 17 -> 18
   strikeout_double_play -> 18 -> 19
   triple -> 19 -> 20
   triple_play -> 20 -> 21
   truncated_pa -> 21 -> 22
   walk -> 22 -> 23


   '''
x_num_indices = [8, 9, 16, 17, 18, 19, 20]
x_low_card_cats_indices = [0, 1, 2, 3, 4, 5, 7, 10, 11, 14]
x_inning_index = 6
x_pitcher_index = 12
x_batter_index = 13
x_prev_pitch_index = 15
x_prev_desc_index = 22
x_prev_event_index = 21

torch.serialization.add_safe_globals([LabelEncoder])
transformers = torch.load(r"C:\Users\Richard\Documents\SEG4300\Project\SEG4300-Project\Code\saved_models\transformers.pt", weights_only=False)
label_encoders_X = transformers["label_encoders_X"]
scaler_X = transformers["scaler_X"]
le_Y_type = transformers["le_Y_type"]
le_Y_result_desc = transformers["le_Y_result_desc"]
le_Y_result_event = transformers["le_Y_result_event"]
scaler_Y_cont = transformers["scaler_Y_cont"]

In [ ]:
def load_trained_model(model_path: r"C:\Users\Richard\Documents\SEG4300\Project\SEG4300-Project\Code\saved_models\best_model_sim.pth"):
    """
    Loads the trained MultiHeadLSTM (or your variant) model from disk.
    If you saved encoders/scalers, load them here as well.
    """
    checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
    model_state = checkpoint['model_state']
    
    # Re-instantiate the model with the same constructor args
    # Adjust the constructor args to match what you used during training
    model = MultiHeadLSTM(
        num_numeric_features=7,  
        num_pitchers=879, 
        num_batters=3096,
        num_prev_descriptions=12,
        num_prev_events=3,
        num_prev_pitch_types=16,
        num_low_card_cats=10,
        num_innings=20,
        inning_emb_dim=4,
        pitcher_emb_dim=30,
        batter_emb_dim=50,
        prev_description_emb_dim=4,
        prev_event_emb_dim=2,
        prev_pitch_emb_dim=4,
        hidden_dim=128,
        num_pitch_type_classes=16,
        num_description_classes=14,
        num_event_classes=24,
        cont_dim=5,
        lstm_layers=2,
        dropout=0.2
    )
    model.load_state_dict(model_state)
    model.eval()
    
    # If you saved encoders, scalers, etc.
    # stand_enc = checkpoint['stand_enc']
    # pthrows_enc = checkpoint['pthrows_enc']
    # ...
    
    return model

SyntaxError: expected argument value expression (2699412148.py, line 12)

In [ ]:
def transform_inference_x(
    raw_features: list,
    label_encoders_X: dict,
    scaler_X,
    num_indices: list,
    cat_indices: list
):
    """
    Converts a single row of raw features (same order & size as training)
    into encoded & scaled values.
    """
    # Make a 1-row DataFrame
    df = pd.DataFrame([raw_features])  # shape (1, n_columns)
    
    # 1) Label-encode any categorical columns
    for col_idx in cat_indices:
        col_name = df.columns[col_idx]
        le = label_encoders_X[col_name]
        df.iloc[:, col_idx] = le.transform(df.iloc[:, col_idx])
        #increment by 1 due to padding
        df.iloc[:, col_idx] = df.iloc[:, col_idx] + 1
    
    # 2) Scale numeric columns
    numeric_part = df.iloc[:, num_indices].values  # shape (1, len(num_indices))
    numeric_scaled = scaler_X.transform(numeric_part)
    df.iloc[:, num_indices] = numeric_scaled
    
    return df.iloc[0].values  # shape => (n_columns,)

In [ ]:
def build_first_pitch_input(
    game_state, 
    prev_pitch_info,
    label_encoders_X,
    scaler_X,
    num_indices,
    cat_indices
):

    raw_features = [
        game_state['balls'],
        game_state['strikes'],
        game_state['outs_when_up'],
        game_state['on_1b'],
        game_state['on_2b'],
        game_state['on_3b'],
        game_state['inning'], 
        game_state['inning_topbot'],
        game_state['home_score'],
        game_state['away_score'],
        game_state['stand'],
        game_state['pthrows'],
        game_state['pitcher_id'],
        game_state['batter_id'],
        game_state['isfirstpitch'],
        prev_pitch_info.get('pitch_type_idx', 0),
        prev_pitch_info.get('pitch_cont', [np.float64(90.3), np.float64(2274.0), np.float64(6.3), np.float64(0.04), np.float64(2.28)])[0],
        prev_pitch_info.get('pitch_cont', [np.float64(90.3), np.float64(2274.0), np.float64(6.3), np.float64(0.04), np.float64(2.28)])[1],
        prev_pitch_info.get('pitch_cont', [np.float64(90.3), np.float64(2274.0), np.float64(6.3), np.float64(0.04), np.float64(2.28)])[2],
        prev_pitch_info.get('pitch_cont', [np.float64(90.3), np.float64(2274.0), np.float64(6.3), np.float64(0.04), np.float64(2.28)])[3],
        prev_pitch_info.get('pitch_cont', [np.float64(90.3), np.float64(2274.0), np.float64(6.3), np.float64(0.04), np.float64(2.28)])[4],
        prev_pitch_info.get('event_idx', 0),
        prev_pitch_info.get('desc_idx', 0),
    ]

    # 2) Apply transform_inference_x to get a single encoded/scaled row
    processed_row = transform_inference_x(
        raw_features,
        label_encoders_X,
        scaler_X,
        num_indices,
        cat_indices
    )
    # processed_row is shape => (n_columns,)

    # 3) Convert to a PyTorch tensor of shape (1,1,n_columns)
    processed_tensor = torch.tensor(processed_row, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    # shape => (batch=1, seq=1, n_columns)

    # 4) Now slice out the pieces your model expects
    # e.g. 
    x_num = processed_tensor[:, :, x_num_indices]
    x_low_card_cats = processed_tensor[:, :, x_low_card_cats_indices]
    x_inning = processed_tensor[:, :, x_inning_index].long()
    x_pitcher = processed_tensor[:, :, x_pitcher_index].long()
    x_batter = processed_tensor[:, :, x_batter_index].long()
    x_prev_pitch = processed_tensor[:, :, x_prev_pitch_index].long()
    x_prev_desc  = processed_tensor[:, :, x_prev_desc_index].long()
    x_prev_event = processed_tensor[:, :, x_prev_event_index].long()

    lengths = torch.tensor([1], dtype=torch.long)

    return {
        'x_num': x_num,
        'x_low_card_cats': x_low_card_cats,
        'x_inning': x_inning,
        'x_pitcher': x_pitcher,
        'x_batter': x_batter,
        'x_prev_pitch': x_prev_pitch,
        'x_prev_desc': x_prev_desc,
        'x_prev_event': x_prev_event,
        'lengths': lengths
    }


In [ ]:
def build_single_step_input(game_state, prev_pitch_info):
    """
    Construct single-step model input from the current game state and previous pitch info.
    Returns a dict of the relevant PyTorch tensors:
      {
        'x_num': shape (1,1,num_x_num),
        'x_low_card_cats': shape (1,1,num_low_card),
        'x_inning': shape (1,1),
        'x_pitcher': shape (1,1),
        'x_batter': shape (1,1),
        'x_prev_pitch': shape (1,1),
        'x_prev_desc': shape (1,1),
        'x_prev_event': shape (1,1),
        'lengths': shape (1,)
      }
    """
    # 1) numeric features array
    # Must match the order in x_num_indices
    # Example: let's say x_num_indices correspond to [balls, strikes, outs, scoreboard, etc.]
    numeric_array = np.array([[
        scaler_X.transform(game_state['home_score']),
        scaler_X.transform(game_state['away_score']),
        prev_pitch_info.get('pitch_cont', 0), #should be scaled atp since from model output
        prev_pitch_info.get('pitch_cont', 1),
        prev_pitch_info.get('pitch_cont', 2),
        prev_pitch_info.get('pitch_cont', 3),
        prev_pitch_info.get('pitch_cont', 4),
    ]], dtype=np.float32)  # shape (1,1,num_x_num)

    # 2) Low-card categorical features
    # For example, if x_low_card_cats_indices are [stand, p_throws, inning_topbot] or something
    # We'll just do a placeholder for demonstration
    low_card_array = np.array([[
        game_state['balls'] + 1,
        game_state['strikes'] + 1,
        game_state['outs_when_up'] + 1,
        game_state['on_1b'] + 1,
        game_state['on_2b'] + 1,
        game_state['on_3b'] + 1,
        game_state['inning_topbot'] + 1,
        game_state['stand'] + 1,
        game_state['pthrows'] + 1,
        game_state['isfirstpitch'] + 1,
    ]], dtype=np.float32)  # shape (1,1,num_low_card)

    # 3) Single-step integer features
    inning_array  = np.array([[game_state['inning'] + 1]], dtype=np.long)  # shape (1,1)
    pitcher_array = np.array([[game_state['pitcher_id'] + 1]], dtype=np.long)
    batter_array  = np.array([[game_state['batter_id'] + 1]], dtype=np.long)

    # If no previous pitch, we set them to 0 or some dummy
    prev_pitch_type = prev_pitch_info.get('pitch_type_idx', 11)
    prev_pitch_desc = prev_pitch_info.get('desc_idx', 1)
    prev_pitch_event= prev_pitch_info.get('event_idx', 2)

    x_prev_pitch_array = np.array([[prev_pitch_type]], dtype=np.long)
    x_prev_desc_array  = np.array([[prev_pitch_desc]], dtype=np.long)
    x_prev_event_array = np.array([[prev_pitch_event]], dtype=np.long)

    # 4) lengths = [1]
    lengths = torch.tensor([1], dtype=torch.long)

    # Convert all to tensors
    return {
        'x_num': torch.tensor(numeric_array, dtype=torch.float32),
        'x_low_card_cats': torch.tensor(low_card_array, dtype=torch.float32),
        'x_inning': torch.tensor(inning_array, dtype=torch.long),
        'x_pitcher': torch.tensor(pitcher_array, dtype=torch.long),
        'x_batter': torch.tensor(batter_array, dtype=torch.long),
        'x_prev_pitch': torch.tensor(x_prev_pitch_array, dtype=torch.long),
        'x_prev_desc': torch.tensor(x_prev_desc_array, dtype=torch.long),
        'x_prev_event': torch.tensor(x_prev_event_array, dtype=torch.long),
        'lengths': lengths
    }

In [ ]:
def simulate_pitch(model, game_state, prev_pitch_info):
    """
    Simulate a single pitch using the model.
    
    Args:
        model: Your trained LSTM model.
        game_state: Dictionary holding the current context (balls, strikes, outs, base-runners, inning, etc.).
        prev_pitch_info: Dictionary (or list) with info about the last pitch (pitch type, speed, spin, result, etc.).
    
    Returns:
        pitch_prediction: e.g. {
            "pitch_type_idx": ...,
            "pitch_cont": [...],
            "desc_idx": ...,
            "event_idx": ...
        }
    """
    # 1) Build model input for this pitch
    #    In practice, you have to replicate the same format you used during training.
    #    For example, if your model expects a single sequence of length=1:

    if game_state['isfirstpitch'] == 1:
        single_input = build_first_pitch_input(game_state, prev_pitch_info) #encode and scales
    else:
        single_input = build_single_step_input(game_state, prev_pitch_info) #should be encoded and scaled

    x_num = single_input['x_num']
    x_low_card_cats = single_input['x_low_card_cats']
    x_pitcher = single_input['x_pitcher']
    x_batter = single_input['x_batter']
    x_inning = single_input['x_inning']
    x_prev_pitch = single_input['x_prev_pitch']
    x_desc = single_input['x_prev_desc']
    x_event = single_input['x_prev_event']
    lengths = single_input['lengths']
    
    # 2) Run the model forward
    with torch.no_grad():
        (pitch_type_logits,
         pitch_cont_values,
         pitch_desc_logits,
         pitch_event_logits) = model(
            x_num,
            x_low_card_cats,
            x_inning,
            x_pitcher,
            x_batter,
            x_desc,
            x_event,
            x_prev_pitch,
            lengths
        )
    
    # pitch_type_logits shape => (1, 1, num_pitch_type_classes)
    # pitch_cont_values => (1, 1, cont_dim)
    # pitch_desc_logits => (1, 1, num_description_classes)
    # pitch_event_logits => (1, 1, num_event_classes)
    
    pitch_type_idx   = torch.argmax(pitch_type_logits[0, 0]).item()
    pitch_cont_array = pitch_cont_values[0, 0].cpu().numpy()
    pitch_desc_idx   = torch.argmax(pitch_desc_logits[0, 0]).item()
    pitch_event_idx  = torch.argmax(pitch_event_logits[0, 0]).item()
    
    return {
        "pitch_type_idx": pitch_type_idx,
        "pitch_cont": pitch_cont_array.tolist(),
        "desc_idx": pitch_desc_idx,
        "event_idx": pitch_event_idx
    }

In [ ]:
# NEED TO ADD A BUNCH OF GAME LOGIC
# UPDATE WITH ENCODED PADDED VALUES 
# should adjust balls, strikes, outs_when_up, bases, score
def update_game_state(game_state, pitch_prediction):
    """
    Update the game state after receiving a predicted pitch outcome.
    For example:
      - If desc_idx => 'ball', increment ball count
      - If desc_idx => 'foul', increment strikes unless already 2
      - If event_idx => 'single', runner(s) move, at-bat ends
      - etc.

    This is domain logic that you'll define yourself.
    
    Returns:
        updated_game_state, at_bat_over (bool)
    """

    if game_state['inning_topbot'] == 0:
        home = False
    else:
        home = True

    # Example simplified logic:
    at_bat_over = False
    out = False
    
    # Suppose "desc_idx" 1 => "called_strike", 2 => "ball", 3 => "foul", etc.
    # Suppose "event_idx" 1 => "in_progress", 2 => "strikeout", 3 => "single", etc.
    desc_idx = pitch_prediction["desc_idx"]
    event_idx = pitch_prediction["event_idx"]

    #BUILD USING ENCODED VALUES (WITH THE INCREMENT FROM DATALOADER PADDING)
    
    if desc_idx in [1, 2, 10]:  # "ball"
        game_state['balls'] += 1
        if game_state['balls'] >= 4:
            # walk => at-bat ends
            at_bat_over = True
            out = False
    
    elif desc_idx in [3, 4, 5, 6, 7, 11, 12, 13]:  # "called_strike" or "foul"
        if desc_idx == 5 and game_state['strikes'] == 2:
            game_state['strikes'] = game_state['strikes']
        else:
            game_state['strikes'] += 1
            
        if game_state['strikes'] >= 3:
            # strikeout => at-bat ends
            at_bat_over = True
            out = True

    # If event indicates a final result (single, double, HR, etc.)

    #NEED TO GET Y INDICES!!!!
    # SINGLE OUT (no runner advance)
    # to handle: [, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
    if event_idx in [2, 6, ]:
        at_bat_over = True
        out = True
    #SINGLE OUT (runner advances)
    elif event_idx in [7, ]:
        if game_state['on_3b'] > 0:
            game_state['on_3b'] = game_state['on_3b'] - 1
            if home:
                game_state['home_score'] = game_state['home_score'] + 1
            else:
                game_state['away_score'] = game_state ['away_score'] + 1
        if game_state['on_2b'] > 0:
            game_state['on_2b'] = game_state['on_1b'] - 1
            game_state['on_3b'] = game_state['on_3b'] + 1
        if game_state['on_1b'] > 0:
            game_state['on_1b'] = game_state['on_1b'] - 1
            game_state['on_2b'] = game_state['on_2b'] + 1

        at_bat_over = True
        out = True
    #HITS
    elif event_idx in []:


    return game_state, at_bat_over, out

In [ ]:
def simulate_at_bat(model, init_game_state):
    """
    Simulate a single at-bat starting from init_game_state.
    
    Returns:
        final_game_state
        pitch_sequence: list of pitch predictions
    """
    game_state = dict(init_game_state)
    pitch_sequence = []
    
    # non encoded / padded
    prev_pitch_info = {
        "pitch_type_idx": 10,
            "desc_idx": 0,
            "event_idx": 1
    }
    game_state['balls'] = 0
    game_state['strikes'] = 0
    game_state['isfirstpitch'] = 1
    
    # Keep simulating pitches until the at-bat ends
    for pitch_num in range(20):  # safety limit in case something gets stuck
        # 1) Predict the next pitch
        pitch_pred = simulate_pitch(model, game_state, prev_pitch_info)
        pitch_sequence.append(pitch_pred)
        game_state ['isfirstpitch'] = 0

        # 2) Update the game state
        updated_state, at_bat_over, out = update_game_state(game_state, pitch_pred)
        game_state = updated_state

        # 3) Set up "previous pitch" info for next iteration
        prev_pitch_info = {
            "pitch_type_idx": pitch_pred["pitch_type_idx"],
            "pitch_cont": pitch_pred["pitch_cont"],
            "desc_idx": pitch_pred["desc_idx"],
            "event_idx": pitch_pred["event_idx"]
        }
        
        if at_bat_over:
            break
    
    # Return the final game state, plus the pitch-by-pitch predictions
    return game_state, pitch_sequence, out

In [ ]:
#MIGHT NEED TO ADAPT TO FORCE TO 5 INNINGS BUT IDK
#CHANGE TO FEED CORRECT PITCHER AND BATTERS FOR TOP/BOT OF INNING
#THIS IS WHERE WE UPDATE PITCHER / BATTER / STAND / PTHROWS
def simulate_game(model):
    """
    Example function to simulate a full game. 
    This is extremely simplified; real logic is more complex.
    """
    # Initialize a simple game state
    #GAME STATE WITHOUT PREV PITCH
    # INITIAL SHOULD BE NON ENCODED
    game_state = {
        "inning": 0, 
        "inning_topbot": 0,  # 0 => top, 1 => bottom
        "outs_when_up": 0,
        "balls": 0,
        "strikes": 0,
        "home_score": 0,
        "away_score": 0,
        "pitcher_id": ,
        "batter_id": ,
        "on_1b": 0,
        "on_2b": 0,
        "on_3b": 0,
        "isfirstpitch": 1
    }
    
    # Suppose we want to simulate 9 innings, or until some condition
    game_summary = []
    
    for inning_i in range(9):
        for half in [0, 1]:  # top/bot
            game_state['inning'] = inning_i
            game_state['inning_topbot'] = half
            # We'll pretend we have 3 outs per half, ignoring base-runner logic for simplicity
            game_state['outs_when_up'] = 0 #starts non encoded but will be updated at first pitch for model
            game_state['on_1b'] = 0
            game_state['on_2b'] = 0
            game_state['on_3b'] = 0

            # Simple loop: each half inning we do a certain number of at-bats
            while game_state['outs_when_up'] < 3:
                # simulate an at-bat
                final_state, pitch_seq, out = simulate_at_bat(model, game_state)

                #update outs_when_up!!! has to happen in this code
                
                # If an at-bat ends in certain event => increment outs, etc.
                # For now, let's say if we see "strikeout," we do outs += 1
                # You can do that in your 'update_game_state' or after the at-bat.

                # We'll just record the pitch sequence
                game_summary.append({
                    "inning": inning_i,
                    "topbot": half,
                    "pitch_sequence": pitch_seq
                })

                # Check if the at-bat ended in a strikeout
                # We'll do a naive approach: if last pitch desc_idx = "strikeout" => outs++
                #WHY NOT JUST ACCESS FROM FINAL_STATE?? ITS THE UDPATED GAME_STATE
                last_pitch = pitch_seq[-1]
                if last_pitch["event_idx"] == 2:  # pretend 2 => "strikeout"
                    game_state['outs_when_up'] += 1
                
                # If the at-bat ended in a walk or hit => also end the at-bat
                # Potentially update runners or scores
                
                # If 3 outs => this half-inning is over
                # incremented
                if game_state['outs_when_up'] >= 3:
                    break
    
    return game_summary

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Richard\Documents\SEG4300\Project\SEG4300-Project\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3278, in run_cell_async
  File "c:\Users\Richard\Documents\SEG4300\Project\SEG4300-Project\.venv\Lib\site-packages\IPython\core\compilerop.py", line 86, in ast_parse
  File "C:\Users\Richard\AppData\Local\Temp\ipykernel_25256\3551678892.py", line 18
    "pitcher_id": ,
                ^
SyntaxError: expression expected after dictionary key and ':'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Richard\Documents\SEG4300\Project\SEG4300-Project\.venv\Lib\site-packages\pygments\styles\__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Richard\Documents\SEG4300\Project\SEG4300-P

In [ ]:
if __name__ == "__main__":
    # Example usage:
    model_path = "path/to/saved_model.pt"
    model = load_trained_model(model_path)
    
    game_log = simulate_game(model)
    
    # Print out some summary
    print("Game simulation complete!")
    for half_inning in game_log:
        print(f"Inning={half_inning['inning']+1}, topbot={half_inning['topbot']} => pitch sequence len={len(half_inning['pitch_sequence'])}")